In [1]:
%pip install langchain-core langgraph>0.2.27


Note: you may need to restart the kernel to use updated packages.


The process cannot access the file because it is being used by another process.


In [3]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()


In [4]:
%pip install -qU "langchain[google-genai]"


Note: you may need to restart the kernel to use updated packages.


In [5]:
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Ade from Lagos Nigeria")])


AIMessage(content="Hi Ade! It's great to meet you. How can I help you today? What are you interested in talking about or learning about? I'm happy to assist with a wide range of topics. Let me know!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0cc43a64-abf2-4d9f-9f34-0fa4468a84b9-0', usage_metadata={'input_tokens': 9, 'output_tokens': 47, 'total_tokens': 56, 'input_token_details': {'cache_read': 0}})

Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/multipart (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C37F8D7250>: Failed to resolve \'api.smith.langchain.com\' ([Errno 11001] getaddrinfo failed)"))'))
Content-Length: 1475
API Key: lsv2_********************************************actrace=93a7cc54-eb7c-404c-982e-88d73c9870af,id=93a7cc54-eb7c-404c-982e-88d73c9870af; trace=93a7cc54-eb7c-404c-982e-88d73c9870af,id=6b03b08a-7577-424b-b030-275c8dc0b305; trace=93a7cc54-eb7c-404c-982e-88d73c9870af,id=26a8ef50-6320-40d9-8331-be2cd2b51295
Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. 

In [7]:
model.invoke([HumanMessage(content="What is my name?")])

AIMessage(content='I am an AI and do not have access to your personal information. Therefore, I do not know your name.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f0699b1a-2973-4869-b4b4-de2b2d6f3f5f-0', usage_metadata={'input_tokens': 5, 'output_tokens': 24, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})

In [8]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Ade from Lagos Nigeria"),
        AIMessage(content="Hello Ade! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)


AIMessage(content='Your name is Ade, from Lagos, Nigeria. You told me that in your first message!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a77c2c3f-937e-4452-af92-25ab863399d6-0', usage_metadata={'input_tokens': 25, 'output_tokens': 20, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema = MessagesState)

# Define the function that calls te model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return{"messages" : response}

# Define the (single) node in the graph
# # Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory) 

In [11]:
config = {"configurable": {"thread_id": "abc123"}}


In [12]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised RetryError: Timeout of 600.0s exceeded, last exception: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: getaddrinfo: WSA Error (No such host is known.
 -- 11001).


RetryError: Timeout of 600.0s exceeded, last exception: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: getaddrinfo: WSA Error (No such host is known.
 -- 11001)